In [54]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-21-openjdk-amd64'

In [55]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
import h3

## 1- Initialize spark

In [56]:
def initialize_spark(app_name: str = "AllPairsShortestPath") -> SparkSession:
    """
    Initializes and returns a SparkSession.
    """
    spark = (
        SparkSession.builder.appName(app_name)
        .config("spark.driver.memory", "8g")
        .getOrCreate()
    )
    spark.sparkContext.setCheckpointDir("checkpoints")
    return spark

## 2- Read edges data and initialize shortcuts table

In [57]:
def read_edges(spark: SparkSession, file_path: str) -> DataFrame:
    """
    Reads edges from a CSV file into a DataFrame.
    """
    edges_df = spark.read.csv(file_path, header=True, inferSchema=True)
    edges_df = edges_df.select("id", "incoming_cell", "outgoing_cell", "lca_res")
    return edges_df

In [58]:
def initial_shortcuts_table(spark: SparkSession, file_path: str, edges_cost_df: DataFrame) -> DataFrame:
    """
    Creates the initial shortcuts table from the edges DataFrame.
    """
    shortcuts_df = spark.read.csv(file_path, header=True, inferSchema=True)
    shortcuts_df = shortcuts_df.select("incoming_edge", "outgoing_edge")
    # Add new via_edge column
    shortcuts_df = shortcuts_df.withColumn("via_edge", F.col("outgoing_edge"))
    
    # shortcuts_df.show(5)
    shortcuts_df = shortcuts_df.join(
        edges_cost_df.select("id", "cost"), 
        shortcuts_df.incoming_edge == edges_cost_df.id, 
        "left"
    ).drop(edges_cost_df.id)
   
    return shortcuts_df

## 3- Update edges cost by dummy function

In [59]:
@F.udf(returnType=DoubleType())
def dummy_cost(length, maxspeed):
    """Calculates cost based on length and maxspeed."""
    if maxspeed <= 0:
        return float('inf')  # Return infinity for invalid speeds
    # Example calculation: Cost is proportional to length and inversely proportional to speed
    return float(length) / float(maxspeed)

def update_dummy_costs_for_edges(spark: SparkSession, file_path: str, edges_df: DataFrame):
    """
    Adds a dummy cost column to the edges DataFrame.
    """
    edges_df_cost = spark.read.csv(file_path, header=True, inferSchema=True).select("id", "length", "maxspeed")
    edges_df_cost = edges_df_cost.withColumn(
        "cost", 
        dummy_cost(F.col("length"), F.col("maxspeed"))
    )
    edges_df = edges_df.drop("cost")  # Remove existing cost column if present
    edges_df = edges_df.join(edges_df_cost.select("id", "cost"), on="id", how="left")
    return edges_df

## 4- Add info "via_cell", "via_res" and "lca_res" to shortcuts table

In [60]:
# LCA - Lowest Common Ancestor resolution
@F.udf(StringType())
def find_LCA(cell1, cell2):
    cell1_res = h3.get_resolution(cell1)
    cell2_res = h3.get_resolution(cell2)
    lca_res = min(cell1_res, cell2_res)
    while h3.cell_to_parent(cell1, lca_res) != h3.cell_to_parent(cell2, lca_res) and lca_res > 0:
        lca_res -= 1
    mycell = h3.cell_to_parent(cell1, lca_res)
    if mycell == h3.cell_to_parent(cell2, lca_res):
        return mycell
    else:
        return None  # maybe return 0 is better option

# Resolution of a cell
@F.udf(IntegerType())  
def find_resolution(cell):
    if cell is None:
        return -1
    return h3.get_resolution(cell)

def add_info_for_shortcuts(spark: SparkSession, shortcuts_df: DataFrame, edges_df: DataFrame) -> DataFrame:
    """
    Adds additional information to the shortcuts DataFrame.
    Add incoming_cell and lca_res from incoming_edge to shortcuts_df
    Add outgoing_cell and lca_res from outgoing_edge to shortcuts_df
    """
    
    # Drop existing columns if present: lca_res, via_cell, via_res
    for col in ["lca_res", "via_cell", "via_res"]:
        if col in shortcuts_df.columns:
            shortcuts_df = shortcuts_df.drop(col)
            
    # Join to get incoming_cell and lca for incoming_edge
    shortcuts_df = shortcuts_df.join(
        edges_df.select(
            F.col("id").alias("incoming_edge_id"),
            F.col("incoming_cell").alias("incoming_cell_in"),
            F.col("lca_res").alias("lca_res_in")
        ),
        shortcuts_df.incoming_edge == F.col("incoming_edge_id"),
        "left"
    ).drop("incoming_edge_id")
    
    # Join to get outgoing_cell and lca for outgoing_edge
    shortcuts_df = shortcuts_df.join(
        edges_df.select(
            F.col("id").alias("outgoing_edge_id"),
            F.col("outgoing_cell").alias("outgoing_cell_out"),
            F.col("lca_res").alias("lca_res_out")
        ),
        shortcuts_df.outgoing_edge == F.col("outgoing_edge_id"),
        "left"
    ).drop("outgoing_edge_id")
    
    # Add lca_res column as the maximum of lca_res_in and lca_res_out
    shortcuts_df = shortcuts_df.withColumn(
        "lca_res",
        F.greatest(F.col("lca_res_in"), F.col("lca_res_out"))
    )
    # Drop intermediate lca_res_in and lca_res_out columns
    shortcuts_df = shortcuts_df.drop("lca_res_in", "lca_res_out")
    
    # Add via_cell column as LCA of incoming_cell_in and outgoing_cell_out
    shortcuts_df = shortcuts_df.withColumn(
        "via_cell", find_LCA(F.col("incoming_cell_in"), F.col("outgoing_cell_out"))
    )   
    shortcuts_df = shortcuts_df.withColumn(
        "via_res", find_resolution(F.col("via_cell"))
    )
    # Drop intermediate incoming_cell_in and outgoing_cell_out columns
    shortcuts_df = shortcuts_df.drop("incoming_cell_in", "outgoing_cell_out")   
    
    return shortcuts_df

## 5- Filter shortcuts table based on current resolution

In [61]:
def filter_shortcuts_by_resolution(shortcuts_df: DataFrame, current_res: int) -> DataFrame:
    """
    Filters the shortcuts DataFrame based on the current resolution.
    Keeps only the shortcuts where lca_res is greater than or equal to current_res.
    """
    filtered_shortcuts_df = shortcuts_df.filter(
        (F.col("lca_res") <= current_res) & (F.col("via_res") >= current_res)
    )
    return filtered_shortcuts_df

## 6- add cell container for each shortcut based on current resolution

In [62]:
def add_parent_cell_at_resolution(shortcuts_df: DataFrame, current_resolution: int) -> DataFrame:
    """
    Adds a cell container column to shortcuts based on the current resolution.
    
    The cell container is computed as the parent cell of the via_cell at the 
    current resolution level. This allows grouping shortcuts by their spatial
    containment at different H3 resolution levels.
    
    Args:
        shortcuts_df: DataFrame with shortcuts containing via_cell column
        current_resolution: The H3 resolution level to use for cell containers
        
    Returns:
        DataFrame with added 'current_cell' column containing the parent cell
    """
    
    @F.udf(StringType())
    def get_parent_cell(cell, target_resolution):
        """
        Get the parent cell of a given cell at the target resolution.
        Returns None if the cell is None or if the resolution is invalid.
        """
        if cell is None:
            return None
        try:
            cell_res = h3.get_resolution(cell)
            # If current resolution is higher than cell resolution, return the cell itself
            if target_resolution >= cell_res:
                return cell
            # Otherwise, get the parent at the target resolution
            return h3.cell_to_parent(cell, target_resolution)
        except Exception:
            return None
    
    # Add the current_cell column based on via_cell
    shortcuts_df = shortcuts_df.withColumn(
        "current_cell",
        get_parent_cell(F.col("via_cell"), F.lit(current_resolution))
    )
    shortcuts_df = shortcuts_df.drop("lca_res", "via_cell", "via_res")
    # Filter out rows where current_cell is None (invalid cells)
    # shortcuts_df = shortcuts_df.filter(F.col("current_cell").isNotNull())
    
    return shortcuts_df

# 7- shortest path

In [63]:
import pandas as pd

def converging_check(old_paths: DataFrame, new_paths: DataFrame) -> bool:
    """
    Checks if two DataFrames of paths are identical.
    Used to determine convergence in the shortest path algorithm.
    """ 
    #old_paths = old_paths.reset_index(drop=True, inplace=False)
    #new_paths = new_paths.reset_index(drop=True, inplace=False)
    # Join on all columns to see if they match
    
    # Perform a left merge with indicator
    merged_df = pd.merge(new_paths, old_paths, on=["incoming_edge","outgoing_edge","cost"], how='left', indicator=True)

    # Filter for 'left_only' rows
    anti_left_join_df = merged_df[merged_df['_merge'] == 'left_only']

    
    #joined = new_paths.join(
    #    old_paths,
    #    on=["incoming_edge","outgoing_edge","cost"],
    #    how="anti_left"
    #)
    return  len(anti_left_join_df)==0  #(len(joined)==0)
    
def compute_shortest_paths_per_partition(
    df_shortcuts: DataFrame,
    partition_columns: list,
) -> DataFrame:
    """
    Computes all-pairs shortest paths using vectorized pandas operations.
    Uses Spark's applyInPandas to process each partition with optimized pandas code.
    
    This approach is significantly faster than pure Spark SQL for small groups
    because it avoids shuffle operations and uses pandas' C-optimized operations.
    
    Args:
        df_shortcuts: DataFrame with schema (incoming_edge, outgoing_edge, cost, via_edge)
        partition_columns: List of column names to group by
        max_iterations_per_group: Maximum iterations per group (default: group size)
    
    Returns:
        DataFrame with computed shortest paths, preserving partition columns
    """
    
    # Infer the schema for output
    
    output_schema = StructType([
        StructField("incoming_edge", StringType(), False),
        StructField("outgoing_edge", StringType(), False),
        StructField("via_edge", StringType(), False),
        StructField("cost", DoubleType(), False),
    ])
    
    def process_partition_pandas(pdf):
        """
        Process a single partition using vectorized pandas operations.
        This function runs inside each Spark executor on the partition's data.
        """
        import pandas as pd
        
        if len(pdf) == 0:
            return pd.DataFrame(columns=['incoming_edge', 'outgoing_edge','via_edge', 'cost'] )
        
        
        # Start with the input edges as initial paths
        paths = pdf[['incoming_edge', 'outgoing_edge', 'via_edge', 'cost']].copy()
        #paths.reset_index(drop=True, inplace=True)
        
        
        # Remove duplicates, keeping minimum cost
        # No need to remove duplicates
        paths = paths.loc[paths.groupby(['incoming_edge', 'outgoing_edge'])['cost'].idxmin()].reset_index(drop=True)
        
        # Set max iterations
        #max_iters = max_iterations_per_group if max_iterations_per_group else len(pdf)
        
        #for iteration in range(max_iters):
        while True:
            # Vectorized merge: if A->B and B->C exist, create A->C
            # This is equivalent to the self-join in Spark but uses pandas merge
            new_paths = paths.merge(
                paths,
                left_on='outgoing_edge',
                right_on='incoming_edge',
                suffixes=('_L', '_R')
            )
            
            # Calculate new costs and keep the first hop's next_node
            new_paths = new_paths[['incoming_edge_L', 'outgoing_edge_R', 'cost_L', 'cost_R', 'outgoing_edge_L']]
            new_paths['cost'] = new_paths['cost_L'] + new_paths['cost_R']
            new_paths = new_paths.rename(columns={
                'incoming_edge_L': 'incoming_edge',
                'outgoing_edge_R': 'outgoing_edge',
                'outgoing_edge_L': 'via_edge'
            })[['incoming_edge', 'outgoing_edge', 'via_edge', 'cost']]
            
            # Filter out self-loops
            new_paths = new_paths[new_paths['incoming_edge'] != new_paths['outgoing_edge']]
            
            if len(new_paths) == 0:
                break  # No new paths to add
            
            # Combine existing and new paths
            combined = pd.concat([paths, new_paths], ignore_index=True)
            
            # Keep only minimum cost for each (src, dst) pair - vectorized operation
            updated_paths = combined.loc[
                combined.groupby(['incoming_edge', 'outgoing_edge'])['cost'].idxmin()
            ].reset_index(drop=True)
            
            # Check convergence: if no change in paths, we're done
            if converging_check(paths, updated_paths):
                break  # Converged
            
            paths = updated_paths
        
        
        return paths
    
    # Apply the pandas function to each partition group
    result = df_shortcuts.groupBy(partition_columns).applyInPandas(
        process_partition_pandas,
        schema=output_schema
    )
    
    return result

# 8- merge

In [64]:
def merge_shortcuts_to_main_table(
    main_df: DataFrame,
    new_shortcuts: DataFrame,
) -> DataFrame:
    """
    Merges newly computed shortcuts back into the main table.
    
    Strategy:
    1. Remove old shortcuts for the processed partitions
    2. Add new shortcuts
    
    Args:
        main_df: Main shortcut table
        new_shortcuts: Newly computed shortcuts
        partition_columns: Columns used for partitioning
    
    Returns:
        Updated DataFrame with new shortcuts
    """
    
    # Remove old shortcuts for these partitions using left_anti join
    #
    # insteed of using this join function, we can just use the part of main_df that filtered out
    main_df = main_df.select("incoming_edge", "outgoing_edge", "cost", "via_edge")
    new_shortcuts = new_shortcuts.select("incoming_edge", "outgoing_edge", "cost", "via_edge")

    remaining_df = main_df.alias("main").join(
        new_shortcuts.alias("update"),
        on=(
            (F.col("main.incoming_edge") == F.col("update.incoming_edge")) &
            (F.col("main.outgoing_edge") == F.col("update.outgoing_edge"))
        ),
        how="left_anti"
    )    
    
    # Combine with new shortcuts
    updated_df = remaining_df.unionByName(new_shortcuts)
    
    return updated_df

In [65]:
spark = initialize_spark()
edges_df = read_edges(spark, file_path="data/burnaby_driving_simplified_edges_with_h3.csv").cache()
edges_cost_df = update_dummy_costs_for_edges(spark, file_path="data/burnaby_driving_simplified_edges_with_h3.csv", edges_df=edges_df)
shortcuts_df = initial_shortcuts_table(spark, file_path="data/burnaby_driving_edge_graph.csv", edges_cost_df=edges_cost_df)


25/11/18 13:35:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [66]:
for current_resolution in range(14,13,-1):
    shortcuts_df = add_info_for_shortcuts(spark, shortcuts_df, edges_df) 
    shortcuts_df = shortcuts_df.checkpoint().cache()
    shortcuts_df.show(5)
    shortcuts_df_filtered =filter_shortcuts_by_resolution(shortcuts_df,current_res=current_resolution)
    shortcuts_df_with_cell =add_parent_cell_at_resolution(shortcuts_df_filtered, current_resolution)
    shortcuts_df_new = compute_shortest_paths_per_partition( shortcuts_df_with_cell, ["current_cell"])
    #shortcuts_df = merge_shortcuts_to_main_table(shortcuts_df, shortcuts_df_new)
    print(f"number of shortcut in resolution {current_resolution} : {shortcuts_df_new.count()}")


spark.stop()

+--------------------+--------------------+--------------------+------------------+-------+---------------+-------+
|       incoming_edge|       outgoing_edge|            via_edge|              cost|lca_res|       via_cell|via_res|
+--------------------+--------------------+--------------------+------------------+-------+---------------+-------+
|(8703571310, 8703...|(8703571308, 8703...|(8703571308, 8703...|1.7882000000000002|     10|8f28de881760406|     15|
|(7314138295, 6092...|(6092498237, 7314...|(6092498237, 7314...|             0.992|     11|8f28de8886c3b50|     15|
|(9068804417, 9068...|(9068804425, 9463...|(9068804425, 9463...|3.4905999999999997|     11|8f28de8982e6759|     15|
|(9289423408, 9289...|(9289423412, 9289...|(9289423412, 9289...|            2.6675|     11|8f28de1228c9309|     15|
|(416104062, 34741...|(347415013, 25364...|(347415013, 25364...|1.9208666666666665|      9|8f28de898932612|     15|
+--------------------+--------------------+--------------------+--------

number of shortcut in resolution 14 : 97963


In [68]:
spark.stop()

In [ ]:
#my_sh = sh.toPandas()